In [14]:
%matplotlib ipympl
from QDP import qdp, atom
import os.path
import numpy as np
from scipy.stats import poisson
from scipy import optimize
import matplotlib.pyplot as plt
dp = '/home/ebert/jupyter-notebooks/FNode-data'

exclude = [2]
single_only = True

ret = 'retention'
err = 'error'
if single_only:
    ret = 'conditional_retention'
    err = 'conditional_retention_err'

In [168]:
exp_name = ''
# exp_name = ''
if exp_name:
    exp_date = '_'.join(exp_name.split('_')[:3])
    experiment_file = os.path.join(exp_date, exp_name, 'results.hdf5')
else:
    experiment_file = ''
q = qdp.QDP(base_data_path=dp)
q.load_data_file(experiment_file)
print "ivars: {}".format(q.experiments[0]['variable_list'])
roi_map = ['cs', 'rb']
cs_cuts = [[27],[2]]
rb_cuts = [[30],[3]]
marker_styles = [
    dict(color='cornflowerblue', marker='o', markeredgecolor='b', linestyle='None'),
    dict(color='indianred', marker='d', markeredgecolor='r', linestyle='None')
]
q.set_thresholds(cs_cuts, roi=0)
q.set_thresholds(rb_cuts, roi=1)
q.generate_thresholds(loss=False, itr=1)
retention = q.apply_thresholds(exclude_rois=exclude, ncondition=1)
# for l in ['retention', 'ivar', 'error']:
#     retention[l] = np.delete(retention[l],-6,1)
tbl_str = "ivar:\t{0}\tretention:{1} +- {2}"
entries = np.product(retention['retention'].shape[:-1])
for i in range(entries):
    print(tbl_str.format(
        retention['ivar'].reshape((entries,-1))[i][0],
        retention[ret].reshape((entries,-1))[i],
        retention[err].reshape((entries,-1))[i]
    ))
# q.save_experiment_data()
q.experiments[0]['variable_list']
try:
    xlab = q.experiments[0]['variable_desc'][q.experiments[0]['variable_list'][0]]['description']
except IndexError:
    print("Only one iteration")
    xlab = ''

fig, ax = plt.subplots()
for e in range(len(retention['ivar'])):
    for r in range(2):
        ax.errorbar(
            retention['ivar'][e, :, r], retention[ret][e, :, r],
            yerr=retention[err][e, :, r],
            **marker_styles[r]
        )
ax.set_ylim(0, 1.01)
ax.set_xlabel(xlab)
ax.set_ylabel('F=4(2) pop.')

fmts = ['pdf', 'png']
fn = os.path.join(dp, q.experiments[0]['source_path'], 'retention_scan.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

data at: 2018_11_02/2018_11_02_11_49_36_temp-9g-5dbm/results.hdf5
name 'fort_exp' is not defined
name 'rb_uwave_freq' is not defined
ivars: [u'fort_drop_us']
--------------------
--------------------
--------------------
--------------------
Internal work array size computation failed: -5
There may be some issue with your guess: `[2.9592795e-18 0.0000000e+00 0.0000000e+00]`
--------------------
Internal work array size computation failed: -5
There may be some issue with your guess: `[2.9592795e-18 0.0000000e+00 0.0000000e+00]`
--------------------
ivar:	0.0	retention:[0.97682119 0.97916667 0.        ] +- [0.01225937 0.01225937 0.01225937]
ivar:	2.85714285714	retention:[0.98036952 0.94505495 0.        ] +- [0.02008945 0.02008945 0.02008945]
ivar:	5.71428571429	retention:[0.97476341 0.90322581 0.        ] +- [0.02578294 0.02578294 0.02578294]
ivar:	8.57142857143	retention:[0.97746967 0.85714286 0.        ] +- [0.03084984 0.03084984 0.03084984]
ivar:	11.4285714286	retention:[0.94251627 0.

FigureCanvasNbAgg()

for k in ['conditional_retention','conditional_retention_err','ivar','retention','error']:
    retention[k] = np.delete(retention[k],-6,1)

In [169]:
def fnode_release_recapture(t, T_uk, a='cs'):
    wr_um = 2.42
    zr_um = 15.6  # calculated
    pfort_nom = 90.  # 80 mV on detector
    pfort_act = 150. #84.2  # mV on detector
    fort_depth_fac = pfort_act/pfort_nom
    if a == 'cs':
        U_mk = 1.481*fort_depth_fac
        fr_khz = 37.2*np.sqrt(fort_depth_fac)
        fa_khz = 3.74*np.sqrt(fort_depth_fac)
        m_au = 132.905
    else:  # rb
        U_mk = 0.883*fort_depth_fac
        fr_khz = 35.5*np.sqrt(fort_depth_fac)
        fa_khz = 3.57*np.sqrt(fort_depth_fac)
        m_au = 86.909
    n=50000
    try:
        res = np.zeros(len(t))
        for i in xrange(len(t)):
            res[i] = atom.release_recapture(t[i], T_uk, U_mk, wr_um, zr_um, fr_khz, fa_khz, n=n, m_au=m_au)
        return res
    except Exception as e:
        print(e)
        return atom.release_recapture(t, T_uk, U_mk, wr_um, zr_um, fr_khz, fa_khz, n=n, m_au=m_au)

def residuals(x, y, t, e, a='cs'):
    T_uk = x[0]
    f0 = x[1]
    res = (y - f0*fnode_release_recapture(t, T_uk, a=a))*np.power(e,-2)
    #print "T_uk = {}, -> {}".format(T_uk, np.sum(np.power(res, 2)))
    return res

def ls_err(x, y, t, f0, w, a='cs'):
    res = residuals(T_uk, y, t, f0, a=a)
    return np.dot(np.power(res, 2), w)

In [170]:
fig, ax = plt.subplots()
T_opts = []
T_uncerts = []
T_opt = 60.  # initial temperature guess
excludes = []

for e in range(len(retention['ivar'])):
    for r in range(2):
        xs = np.delete(retention['ivar'][e, :, r], excludes)
        ys = np.delete(retention['conditional_retention'][e, :, r], excludes)
        yerrs = np.delete(retention['conditional_retention_err'][e, :, r], excludes)
        ax.errorbar(xs, ys,
                    yerr=yerrs,
                    **marker_styles[r]
        )
        p0 = np.mean(retention['conditional_retention'][e,:2,r])
        guess = (T_opt, p0)
        args = (ys, xs, yerrs, ['cs','rb'][r])
        bounds = ([0,0],[np.inf,1])
        x = optimize.least_squares(
            residuals,
            guess,
            bounds=bounds,
            args=args,
            diff_step=0.05,
            xtol=0.01
        )
        if x['success']:
            T_opt = x['x'][0]
        T_opts.append(T_opt)

        J = x.jac
        res = residuals(x['x'], ys, xs, yerrs)
        ucert = np.sqrt(np.abs(np.linalg.inv(J.T.dot(J)) * (res**2).mean())[0,0])
        T_uncerts.append(ucert)
        print ucert

        pltlim = max(1.05*np.max(xs),35)
        xss = np.linspace(0, pltlim, 30)
        ax.plot(xss, fnode_release_recapture(xss, T_opt, a=['cs','rb'][r])*x['x'][1], 'k-')
        ax.text(1, 0.4-r*0.07, "T = {:.1f} uK".format(T_opt))
    
ax.set_ylim(-0.01, 1.01)
ax.set_xlim(-0.5, pltlim)
ax.set_xlabel(r'ODT Drop Time ($\mu$s)', fontsize=14)
ax.set_ylabel('Retention', fontsize=14)
ax.grid(which='both')
plt.tight_layout()

fmts = ['pdf', 'png', 'eps']
fn = os.path.join(dp, q.experiments[0]['source_path'], 'temperature_fit_w=2.4um_zr=15.6um.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

FigureCanvasNbAgg()

1.7155918793404945
18.395523822413136


In [5]:
fig, ax = plt.subplots()
Ts = [100,100]

for r in range(2):
    xs = np.linspace(5, 600, 30)
    rets = [fnode_release_recapture([6], x, a=['cs','rb'][r])*[0.8,0.9][r] for x in xs]
    ax.plot(xs, rets, 'br'[r]+'-')
    
ax.set_ylim(0, 1.0)
# ax.set_xlim(-0.5, 26)
ax.set_xlabel(r'T ($\mu$K)', fontsize=14)
ax.set_ylabel('Retention', fontsize=14)
ax.grid(which='both')
plt.tight_layout()

FigureCanvasNbAgg()

# Find maximally sensitive drop time

In [74]:
# where can I be most sensitive for a temperature measurement?
fig, ax = plt.subplots()
for r in range(2):
    sensitivity = []
    c = [167, 96][r]
    dropts = np.linspace(5, 60, 31)
    for dropt in dropts:
        ts = np.linspace(c-10, c+10, 5)
        tdata = [fnode_release_recapture(dropt, T, a=roi_map[r]) for T in ts]
        sensitivity.append(np.polyfit(ts, tdata, 1)[0]*1000)
    ax.plot(dropts, sensitivity, 'br'[r]+'-')
    
# ax.set_ylim(0, 1.01)
ax.set_xlabel('drop time (ms)')
ax.set_ylabel('d retention/dT a.u.')

FigureCanvasNbAgg()

object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' has no len()
object of type 'numpy.float64' h

Text(0,0.5,u'd retention/dT a.u.')